In [12]:
""" !pip install datetime
!pip install astroquery
!pip install -q pymcel """


' !pip install datetime\n!pip install astroquery\n!pip install -q pymcel '

In [13]:
import spiceypy as spy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astroquery.jplhorizons import Horizons
from astropy import constants
import datetime


<p>
<img src="https://drive.google.com/uc?id=1ZHUKcwzA5c-PKUwoGmBG3TsHk6TmZ-Y0" align="left" width="250">
</p>
<p>
<h1><b><center>Mecánica Celeste</center></b></h1>
<h2><b><center>Prof. Jorge I. Zuluaga</center></b></h1>
<h2><b><center>Tarea 1</center></b><h2>
<h3><b><center>Constantes de movimiento del sistema solar</center></b><h3>
<h5><center><b>Asignada</b>: 15 de Agosto de 2023</center><h5>
<h5><center><b>Entrega</b>: 29 de Agosto de 2023</center><h5>
</p>

<hr/>
<b>Nombre</b>: Bryan Camilo Restrepo Arcila
<br/>
<b>Cédula</b>:1000567825
<br/>
<b>Última actualización</b>:
<hr/>

## Enunciado

**Objetivo**: El objetivo de esta tarea es poner a prueba los resultados vistos en clase sobre las constantes en el problema de los N-cuerpos, en este caso aplicados al sistema solar.  Para ello usaremos el sistema de datos del JPL Horizons y el paquete `astroquery`.

**Procedimiento**:

Para conseguir el objetivo:

1. Obtener las masas, posiciones y velocidades de los planetas del sistema solar (8 planetas) y del Sol durante 20 años (por ejemplo entre 2000 y 2020), cada 3 meses. Use para ello Astroquery y SPICE (para las masas).

2. Para cada punto en el tiempo calcular:

   - El momentum lineal de cada cuerpo.
   - El momentum angular de cada cuerpo.
   - La energía cinética de cada cuerpo.
   - La energía potencial gravidtacional de todo el sistema.

3. Usando el valor de cada una de las cantidades anteriores vamos a calcular el valor de las cuadraturas del sistema:

   - Calcular el momentum lineal total (3 cuadraturas).
   - Calcular el momentum angular total (3 cuadraturas).
   - Calcular la energía total total (1 cuadratura).

   Es muy importante tener en cuenta que el valor de cada una de las cuadraturas puede ser inmenso (las masas del sistema solar son inmensas). Por lo tanto, debemos dividir cada cuadratura por el valor característico de. para ello calcule el valor promedio del momentum lineal, angular y energía cinética del Sol.

4. Hacer gráficos de las 10 cuadraturas.

5. El momentum angular total del sistema solar define el Plano de Laplace del sistema. El momentum angular de cada uno de los cuerpos del sistema solar define la inclinación de la órbita osculatriz del cuerpo respecto al plano de Laplace. Calcular, para cada tiempo, el ángulo entre el plano de Laplace y la órbita osculatriz de cada cuerpo (incluyendo el Sol).

Ayuda: el ángulo entre dos vectores se calcula usando la propiedad del producto punto A⋅ B = A B cos θ.

6. Haga un gráfico del ángulo de la órbita osculatriz de cada cuerpo como función del tiempo.

7. Calcule el valor promedio de la inclinación orbital de cada planeta en grados respecto al plano invariante de Laplace.

**Resultados esperados**:

- Este notebook con el procedimiento debidamente comentado.

- Gráficos mínimos:
  - Gráfico de las 10 cuadraturas normalizadas (divididas por el valor para el Sol).

  - Gráfico de la inclinación de la órbita osculatriz de cada planeta y el Sol.

- Datos mínimos:
  - Valor de la inclinación orbital de cada planeta

- Gráficos opcionales:
  - Gráfico de las órbitas de los planetas.
  - Gráfico de las magnitudes del momentum lineal y angular total como función del tiempo.

- En caso de utilizar ayudas como `ChatGPT` o códigos de `Stack Over Flow` debe citar debidamente la fuente, pero también entender a cabalidad el código, el profesor puede pedir justificación.

## Solución

## 1. Obtener las masas, posiciones y velocidades de los planetas del sistema solar (8 planetas) y del Sol durante 20 años (por ejemplo entre 2000 y 2020), cada 3 meses. Use para ello Astroquery y SPICE (para las masas).

In [14]:
spy.bodvrd(f'199','GM',1)

(1, array([22031.8685514]))

In [15]:
#Praparemos la información que se pouede obtener de SPICE

#Codigo de la clase poara importar la libreria de SPICE la cual contiene la data necesaria para encontrar la masa de los planetas y el sol
!wget https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/gm_de440.tpc -O gm_de440.tpc
spy.furnsh('gm_de440.tpc')

#Con el siguiente codigo obtenemos la masa de los planetas, y poniendo esta en un datfarame con los planetas como indices
dic={}
for i in range(1,9) :
  n,dato = spy.bodvrd(f'{i}99','GM',1)
  dato=dato/((constants.G.value)/10**9) #G debe de estar en las unidades correctas km^3/(kg*s^2)
  dic[f'{i}99']=dato[0]

#Añadiendo la masa del sol
dic['1099']=constants.M_sun.value

--2023-08-20 21:10:07--  https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/gm_de440.tpc
Resolving naif.jpl.nasa.gov (naif.jpl.nasa.gov)... 

137.78.232.95
Connecting to naif.jpl.nasa.gov (naif.jpl.nasa.gov)|137.78.232.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12708 (12K) [text/plain]
Saving to: ‘gm_de440.tpc’

gm_de440.tpc        100%[===================>]  12.41K  --.-KB/s    in 0.001s  

2023-08-20 21:10:10 (11.1 MB/s) - ‘gm_de440.tpc’ saved [12708/12708]



{'199': 3.301000636980658e+23,
 '299': 4.867305814842006e+24,
 '399': 5.972168399787583e+24,
 '499': 6.416908682663215e+23,
 '599': 1.898124625809005e+27,
 '699': 5.6831737012663016e+26,
 '799': 8.680987154498916e+25,
 '899': 1.0240928854654863e+26,
 '1099': 1.988409870698051e+30}

In [16]:
# Lista de IDs de los planetas
id_planets = list(np.arange(199, 1199, 100))
id_planets.remove(999)  # Quitando a Pluton de la lista

# Diccionario con los nombres de los planetas para una mejor identificación
planet_names = {
    199: 'Mercury',
    299: 'Venus',
    399: 'Earth',
    499: 'Mars',
    599: 'Jupiter',
    699: 'Saturn',
    799: 'Uranus',
    899: 'Neptune',
    1099:'sun'
}

# Inicializar el DataFrame final
df_planets_sun_information = None

for i in id_planets:
    # Extrayendo los datos de los planetas en referencia al baricentro del sistema solar
    information = Horizons(id=str(i), location='0', epochs={'start': '2002-01-01', 'stop': '2022-12-31', 'step': '90d'}).vectors().to_pandas()[['datetime_str', 'x', 'y', 'z', 'vx', 'vy', 'vz']]
    #vamos a agregarle la magnitud de la posición:
    information['|r|'] = np.sqrt(information['x']**2 + information['y']**2 + information['z']**2)
    #Vamos a agregarle la magnitud de la velocidad:
    information['|v|'] = np.sqrt(information['vx']**2 + information['vy']**2 + information['vz']**2)   
    
    #Añadiendo la información de la masa de los planetas y el sol
    information['mass'] = dic[str(i)]

   
    # Cambiar nombres de columnas para identificar a qué planeta pertenecen y agregar unidades correctamente
    columns = {}
    for col in information.columns:
        if col == 'mass':
            columns[col] = f'{planet_names[i]}.{col} (Kg)'
        elif col in ['vx', 'vy', 'vz', '|v|']:
            columns[col] = f'{planet_names[i]}.{col} (au/day)'
        elif col in ['x', 'y', 'z', '|r|']:
            columns[col] = f'{planet_names[i]}.{col} (au)'
        

    information = information.rename(columns=columns)
    
    # Si el DataFrame final está vacío, toca ponerle la información del primer planeta (Mercurio)
    if df_planets_sun_information is None:
        df_planets_sun_information = information
    else:
        # Combinar DataFrames basado en la columna datetime_str
        df_planets_sun_information = df_planets_sun_information.merge(information, on='datetime_str', how='outer')


pd.set_option('display.max_columns', None)
df_planets_sun_information

,datetime_str,Mercury.x (au),Mercury.y (au),Mercury.z (au),Mercury.vx (au/day),Mercury.vy (au/day),Mercury.vz (au/day),Mercury.|r| (au),Mercury.|v| (au/day),Mercury.mass (Kg),Venus.x (au),Venus.y (au),Venus.z (au),Venus.vx (au/day),Venus.vy (au/day),Venus.vz (au/day),Venus.|r| (au),Venus.|v| (au/day),Venus.mass (Kg),Earth.x (au),Earth.y (au),Earth.z (au),Earth.vx (au/day),Earth.vy (au/day),Earth.vz (au/day),Earth.|r| (au),Earth.|v| (au/day),Earth.mass (Kg),Mars.x (au),Mars.y (au),Mars.z (au),Mars.vx (au/day),Mars.vy (au/day),Mars.vz (au/day),Mars.|r| (au),Mars.|v| (au/day),Mars.mass (Kg),Jupiter.x (au),Jupiter.y (au),Jupiter.z (au),Jupiter.vx (au/day),Jupiter.vy (au/day),Jupiter.vz (au/day),Jupiter.|r| (au),Jupiter.|v| (au/day),Jupiter.mass (Kg),Saturn.x (au),Saturn.y (au),Saturn.z (au),Saturn.vx (au/day),Saturn.vy (au/day),Saturn.vz (au/day),Saturn.|r| (au),Saturn.|v| (au/day),Saturn.mass (Kg),Uranus.x (au),Uranus.y (au),Uranus.z (au),Uranus.vx (au/day),Uranus.vy (au/day),Uranus.vz (au/day),Uranus.|r| (au),Uranus.|v| (au/day),Uranus.mass (Kg),Neptune.x (au),Neptune.y (au),Neptune.z (au),Neptune.vx (au/day),Neptune.vy (au/day),Neptune.vz (au/day),Neptune.|r| (au),Neptune.|v| (au/day),Neptune.mass (Kg),sun.x (au),sun.y (au),sun.z (au),sun.vx (au/day),sun.vy (au/day),sun.vz (au/day),sun.|r| (au),sun.|v| (au/day),sun.mass (Kg)
0,A.D. 2002-Jan-01 00:00:00.0000,0.530696,-1.109092,-0.044066,0.022346,0.030571,0.001769,1.230311,0.037909,3.301001e+23,0.212419,-1.693480,-0.011990,0.037428,0.004097,-0.001157,1.706792,0.037670,4.867306e+24,0.000005,-0.000037,-0.000020,0.000165,0.000027,-0.000012,0.000043,0.000168,5.972168e+24,1.456362,-0.339910,-0.018315,0.011739,0.016941,0.000415,1.495615,0.020615,6.416909e+23,-0.773198,4.115578,0.000142,0.009842,0.002152,0.000161,4.187579,0.010076,1.898125e+27,2.899686,7.672785,-0.258620,0.011738,0.004847,0.000184,8.206503,0.012701,5.683174e+26,16.413548,-12.621713,-0.253768,0.019619,0.006194,-0.000029,20.706921,0.020574,8.680987e+25,18.821012,-24.592550,0.056900,0.019798,0.005145,-0.000108,30.968165,0.020456,1.024093e+26,2.513070,-0.860100,-0.170810,0.016803,0.015581,0.002384,2.661667,0.023040,1.988410e+30
1,A.D. 2002-Apr-01 00:00:00.0000,1.340539,0.106918,-0.039994,-0.002045,0.045745,0.002182,1.345391,0.045842,3.301001e+23,1.386339,0.788867,-0.015286,-0.019836,0.028374,0.001057,1.595143,0.034637,4.867306e+24,0.000026,-0.000009,-0.000032,-0.000026,0.000152,-0.000066,0.000043,0.000168,5.972168e+24,1.350435,1.681856,0.022092,-0.016091,0.021654,0.000350,2.157035,0.026981,6.416909e+23,-0.636144,5.138320,0.015608,-0.010316,0.015106,0.000104,5.177572,0.018293,1.898125e+27,3.193027,8.967495,-0.240703,-0.008756,0.018446,0.000137,9.522045,0.020419,5.683174e+26,17.417928,-11.189997,-0.255364,-0.000839,0.020145,-0.000084,20.704238,0.020162,8.680987e+25,19.843084,-23.255951,0.048192,-0.000625,0.019080,-0.000163,30.571058,0.019091,1.024093e+26,3.055378,1.371170,0.053765,-0.008163,0.028174,0.002452,3.349378,0.029435,1.988410e+30
2,A.D. 2002-Jun-30 00:00:00.0000,0.215629,0.980823,-0.034974,-0.020330,0.026970,0.002717,1.004854,0.033884,3.301001e+23,-0.812528,0.742523,0.035047,-0.009646,-0.021264,-0.000670,1.101259,0.023359,4.867306e+24,-0.000004,0.000011,-0.000041,-0.000166,-0.000020,0.000009,0.000043,0.000168,5.972168e+24,-0.941569,2.429316,0.049399,-0.028587,-0.008019,0.000177,2.605872,0.029691,6.416909e+23,-2.397851,5.732875,0.030809,-0.023832,-0.005250,0.000175,6.214217,0.024404,1.898125e+27,1.552976,9.889621,-0.222003,-0.022704,-0.001320,0.000220,10.013272,0.022743,5.683174e+26,16.491178,-10.099053,-0.256882,-0.014772,0.000733,-0.000007,19.339489,0.014790,8.680987e+25,18.937296,-22.261474,0.039484,-0.014521,-0.000346,-0.000088,29.226631,0.014525,1.024093e+26,1.312797,3.058665,0.269841,-0.025297,0.005783,0.002236,3.339413,0.026046,1.988410e+30
3,A.D. 2002-Sep-28 00:00:00.0000,-0.652995,-0.047782,-0.029006,-0.006479,0.012027,0.003207,0.655384,0.014033,3.301001e+23,-0.299122,-0.275845,-0.043070,0.006998,0.002165,0.0

# 2. Para cada punto en el tiempo calcular:

   - El momentum lineal de cada cuerpo.
   - El momentum angular de cada cuerpo.
   - La energía cinética de cada cuerpo.
   - La energía potencial gravitacional de todo el sistema.

In [33]:
#En el siguiente codigo se va a calcular la información anteriormente solicitada según la data del punto 1.

#Para esto tengamos en cuenta que:
#El momento lineal esta dado por: p = m*v
#El momento angular esta dado por: L = r x p como estos vectores son perpendiculares, entonces L = r*p
#La energía cinética esta dada por: E = 1/2*m*v^2
#La energía potencial gravitacional esta dada por: U = -G*M*m/r

#Para esto agreguemos las columnas necesarias en el dataframe anterior y después extraemos solo las columnas necesarias
#Reciclando lo necesario del código anterior

# Lista de IDs de los planetas
id_planets = list(np.arange(199, 1199, 100))
id_planets.remove(999)  # Quitando a Pluton de la lista

# Diccionario con los nombres de los planetas para una mejor identificación
planet_names = {
    199: 'Mercury',
    299: 'Venus',
    399: 'Earth',
    499: 'Mars',
    599: 'Jupiter',
    699: 'Saturn',
    799: 'Uranus',
    899: 'Neptune',
    1099:'sun'
}

# Inicializar el DataFrame final
df_planets_sun_information = None

for i in id_planets:
    # Extrayendo los datos de los planetas en referencia al baricentro del sistema solar
    information = Horizons(id=str(i), location='0', epochs={'start': '2002-01-01', 'stop': '2022-12-31', 'step': '90d'}).vectors().to_pandas()[['datetime_str', 'x', 'y', 'z', 'vx', 'vy', 'vz']]
    #vamos a agregarle la magnitud de la posición:
    information['|r|'] = np.sqrt(information['x']**2 + information['y']**2 + information['z']**2)
    #Vamos a agregarle la magnitud de la velocidad:
    information['|v|'] = np.sqrt(information['vx']**2 + information['vy']**2 + information['vz']**2)   
    
    #Añadiendo la información de la masa de los planetas y el sol
    information['mass'] = dic[str(i)]

    #Añadiendo la información del momento lineal
    information['|p|'] = information['mass']*information['|v|']
    
    #añadiendo la información del momento angular
    information['|L|'] = information['mass']*information['|r|']*information['|v|']

    #Añadiendo la información de la energía cinética
    information['Ec'] = 0.5*information['mass']*information['|v|']**2

    #Añadiendo la información de la energía potencial gravitacional
    information['U'] = -constants.G*information['mass']*constants.M_sun.value/information['|r|']/(1.495978707e+11)**3
    # Cambiar nombres de columnas para identificar a qué planeta pertenecen y agregar unidades correctamente
    
    information=information.drop(['x','y','z','vx','vy','vz','mass','|v|','|r|'],axis=1) #Para que el dataframe no sea tan grande eliminenos las columnas del punto 1
    columns = {}

    for col in information.columns:
        if col == '|p|':
            columns[col] = f'{planet_names[i]}.{col} (Kg au/day)'
        elif col in ['|L|']:
            columns[col] = f'{planet_names[i]}.{col} (Kg au^2/day)'
        elif col in ['Ec']:
            columns[col] = f'{planet_names[i]}.{col} (Kg au^2/day^2)'
        elif col in ['U']:
            columns[col] = f'{planet_names[i]}.{col} (Kg au^2/s^2)'

    information = information.rename(columns=columns)
    
    # Si el DataFrame final está vacío, toca ponerle la información del primer planeta (Mercurio)
    if df_planets_sun_information is None:
        df_planets_sun_information = information
    else:
        # Combinar DataFrames basado en la columna datetime_str
        df_planets_sun_information = df_planets_sun_information.merge(information, on='datetime_str', how='outer')



pd.set_option('display.max_columns', None)
df_planets_sun_information


,datetime_str,Mercury.|p| (Kg au/day),Mercury.|L| (Kg au^2/day),Mercury.Ec (Kg au^2/day^2),Mercury.U (Kg au^2/s^2),Venus.|p| (Kg au/day),Venus.|L| (Kg au^2/day),Venus.Ec (Kg au^2/day^2),Venus.U (Kg au^2/s^2),Earth.|p| (Kg au/day),Earth.|L| (Kg au^2/day),Earth.Ec (Kg au^2/day^2),Earth.U (Kg au^2/s^2),Mars.|p| (Kg au/day),Mars.|L| (Kg au^2/day),Mars.Ec (Kg au^2/day^2),Mars.U (Kg au^2/s^2),Jupiter.|p| (Kg au/day),Jupiter.|L| (Kg au^2/day),Jupiter.Ec (Kg au^2/day^2),Jupiter.U (Kg au^2/s^2),Saturn.|p| (Kg au/day),Saturn.|L| (Kg au^2/day),Saturn.Ec (Kg au^2/day^2),Saturn.U (Kg au^2/s^2),Uranus.|p| (Kg au/day),Uranus.|L| (Kg au^2/day),Uranus.Ec (Kg au^2/day^2),Uranus.U (Kg au^2/s^2),Neptune.|p| (Kg au/day),Neptune.|L| (Kg au^2/day),Neptune.Ec (Kg au^2/day^2),Neptune.U (Kg au^2/s^2),sun.|p| (Kg au/day),sun.|L| (Kg au^2/day),sun.Ec (Kg au^2/day^2),sun.U (Kg au^2/s^2)
0,A.D. 2002-Jan-01 00:00:00.0000,1.251363e+22,1.539566e+22,2.371870e+20,-1.063570e+10,1.833496e+23,3.129396e+23,3.453354e+21,-1.130429e+11,1.001221e+21,4.260040e+16,8.392624e+16,-5.563956e+15,1.322835e+22,1.978452e+22,1.363502e+20,-1.700754e+10,1.912544e+25,8.008931e+25,9.635369e+22,-1.796789e+13,7.218209e+24,5.923625e+25,4.583930e+22,-2.745163e+12,1.786015e+24,3.698286e+25,1.837261e+22,-1.661839e+11,2.094834e+24,6.487315e+25,2.142544e+22,-1.310869e+11,4.581205e+28,1.219364e+29,5.277443e+26,-2.961336e+16
1,A.D. 2002-Apr-01 00:00:00.0000,1.513251e+22,2.035914e+22,3.468536e+20,-9.725961e+09,1.685864e+23,2.689195e+23,2.919622e+21,-1.209552e+11,1.001220e+21,4.260037e+16,8.392612e+16,-5.563956e+15,1.731321e+22,3.734521e+22,2.335605e+20,-1.179245e+10,3.472260e+25,1.797788e+26,3.175921e+23,-1.453229e+13,1.160464e+25,1.104999e+26,1.184793e+23,-2.365899e+12,1.750287e+24,3.623836e+25,1.764491e+22,-1.662055e+11,1.955099e+24,5.976944e+25,1.866242e+22,-1.327897e+11,5.852921e+28,1.960365e+29,8.614092e+26,-2.353299e+16
2,A.D. 2002-Jun-30 00:00:00.0000,1.118495e+22,1.123924e+22,1.894926e+20,-1.302201e+10,1.136961e+23,1.252088e+23,1.327921e+21,-1.752002e+11,1.001219e+21,4.260030e+16,8.392586e+16,-5.563956e+15,1.905276e+22,4.964905e+22,2.828523e+20,-9.761311e+09,4.632172e+25,2.878532e+26,5.652164e+23,-1.210804e+13,1.292536e+25,1.294251e+26,1.469821e+23,-2.249833e+12,1.283938e+24,2.483070e+25,9.494863e+21,-1.779342e+11,1.487507e+24,4.347481e+25,1.080310e+22,-1.388980e+11,5.179014e+28,1.729487e+29,6.744632e+26,-2.360321e+16
3,A.D. 2002-Sep-28 00:00:00.0000,4.632204e+21,3.035871e+21,3.250123e+19,-1.996574e+10,3.565239e+22,1.458787e+22,1.305746e+20,-4.715423e+11,1.001219e+21,4.260030e+16,8.392587e+16,-5.563956e+15,1.876696e+22,4.890503e+22,2.744301e+20,-9.761162e+09,4.081978e+25,2.371630e+26,4.389213e+23,-1.295042e+13,9.684172e+24,8.603129e+25,8.250952e+22,-2.535901e+12,1.268487e+24,2.438284e+25,9.267714e+21,-1.790218e+11,1.612154e+24,4.759241e+25,1.268947e+22,-1.375129e+11,3.008411e+28,7.863885e+28,2.275822e+26,-3.015375e+16
4,A.D. 2002-Dec-27 00:00:00.0000,1.004036e+22,9.816711e+21,1.526945e+20,-1.338334e+10,5.566107e+22,3.115348e+22,3.182617e+20,-3.447220e+11,1.001220e+21,4.260037e+16,8.392615e+16,-5.563956e+15,1.604397e+22,3.356387e+22,2.005708e+20,-1.215909e+10,2.292150e+25,1.038522e+26,1.383985e+23,-1.660686e+13,6.752584e+24,5.446603e+25,4.011613e+22,-2.792998e+12,1.752276e+24,3.612932e+25,1.768504e+22,-1.668965e+11,2.076412e+24,6.414397e+25,2.105027e+22,-1.314112e+11,1.612558e+28,3.252268e+28,6.538748e+25,-3.908141e+16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,A.D. 2021-Dec-17 00:00:00.0000,1.290965e+22,1.773114e+22,2.524373e+20,-9.527058e+09,2.609621e+22,8.653431e+21,6.995783e+19,-5.818529e+11,1.001219e+21,4.260033e+16,8.392599e+16,-5.563956e+15,1.930988e+22,4.665360e+22,2.905384e+20,-1.052824e+10,4.024181e+25,2.161996e+26,4.265799e+23,-1.400499e+13,1.217128e+25,1.289282e+26,1.303321e+23,-2.126740e+12,1.295598e+24,2.462286e+25,9.668110e+21,-1.810657e+11,1.850886e+24,5.550164e